# Q-Learning


In [18]:
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import math
import random
from itertools import product, combinations

In [19]:
def getPossibleStates(edges, max_age):
    """
    Returns: list of states, where each state is a tuple of ((edge), age)
    """
    sorted_edges = sorted(tuple(sorted(e)) for e in edges)
    possible_ages = [-1] + list(range(1, max_age + 1))

    all_states = [
        tuple(zip(sorted_edges, age_combo))
        for age_combo in product(possible_ages, repeat=len(sorted_edges))
    ]

    return all_states

def getAgedStates(state, maxAge):
    """
    Returns:
        New state as a tuple of ((edge), age) pairs
    """
    new_state = []
    for edge, age in state:
        if age == -1:
            new_state.append((edge, -1))  # not entangled
        else:
            new_age = age + 1
            if new_age <= maxAge:
                new_state.append((edge, new_age))
            else:
                new_state.append((edge, -1))  # entanglement expired

    return tuple(sorted(new_state))


def generateAllOutcomes(state, pGen):
    empty_edges = [edge for edge, age in state if age == -1]
    outcomes = []

    for pattern in product([0, 1], repeat=len(empty_edges)):
        prob = 1.0
        new_state = []

        # First, update the empty edges based on entanglement outcomes
        outcome_map = {}
        for (edge, outcome) in zip(empty_edges, pattern):
            if outcome:
                prob *= pGen
                outcome_map[edge] = 1
            else:
                prob *= (1 - pGen)
                outcome_map[edge] = -1

        # Now build the full new state
        for edge, age in state:
            if edge in outcome_map:
                new_state.append((edge, outcome_map[edge]))
            else:
                new_state.append((edge, age))  # unchanged

        outcomes.append((tuple(sorted(new_state)), prob))

    return outcomes

def generateAllSwappingOutcomes(state, goalEdges, pSwap):
    """
    Generate all possible outcomes from swapping operations or doing nothing.
    Can attempt multiple non-overlapping swaps simultaneously.
    """
    def find_path(current, target, visited):
        if current == target:
            return [current]
        visited.add(current)
        for next_node in graph.get(current, []):
            if next_node not in visited:
                path = find_path(next_node, target, visited)
                if path:
                    return [current] + path
        return None

    # First find all possible individual swap attempts
    swap_attempts = []
    entangled_edges = [(edge, age) for edge, age in state if age > 0]
    
    # Create graph once for all path finding
    graph = {}
    for (edge, _) in entangled_edges:
        if edge[0] not in graph: graph[edge[0]] = []
        if edge[1] not in graph: graph[edge[1]] = []
        graph[edge[0]].append(edge[1])
        graph[edge[1]].append(edge[0])
    
    # Find all possible paths for each goal
    for goal_edge in goalEdges:
        start, end = goal_edge
        path = find_path(start, end, set())
        
        if path and len(path) > 1:
            path_edges = list(zip(path[:-1], path[1:]))
            used_edges = []
            for p_edge in path_edges:
                for e, age in entangled_edges:
                    if (e[0] == p_edge[0] and e[1] == p_edge[1]) or \
                       (e[0] == p_edge[1] and e[1] == p_edge[0]):
                        used_edges.append((e, age))
            
            swap_attempts.append({
                'goal': goal_edge,
                'used_edges': used_edges,
                'num_swaps': len(path_edges) - 1
            })
    
    outcomes = []
    
    # Case 1: Do nothing
    outcomes.append((state, 1.0, None))
    
    # Case 2: Try all possible combinations of non-overlapping swaps
    for r in range(1, len(swap_attempts) + 1):
        for attempt_combo in combinations(swap_attempts, r):
            # Check if attempts are compatible (don't share edges)
            all_used_edges = set()
            edge_overlap = False
            for attempt in attempt_combo:
                attempt_edges = set(e for e, _ in attempt['used_edges'])
                if any(e in all_used_edges for e in attempt_edges):
                    edge_overlap = True
                    break
                all_used_edges.update(attempt_edges)
            
            if edge_overlap:
                continue
                
            # Generate all success/failure combinations for this set of attempts
            for success_pattern in product([True, False], repeat=len(attempt_combo)):
                new_state = []
                total_prob = 1.0
                achieved_goals = []
                
                # Calculate probability and track achieved goals
                for attempt, succeeded in zip(attempt_combo, success_pattern):
                    prob = pSwap ** attempt['num_swaps'] if succeeded else \
                          (1 - pSwap ** attempt['num_swaps'])
                    total_prob *= prob
                    if succeeded:
                        achieved_goals.append((attempt['goal'], True))
                    else:
                        achieved_goals.append((attempt['goal'], False))
                
                # Create new state
                for edge, age in state:
                    if any(edge == e for attempt in attempt_combo 
                          for e, _ in attempt['used_edges']):
                        new_state.append((edge, -1))  # Used edges become unentangled
                    else:
                        new_state.append((edge, age))  # Other edges remain unchanged
                
                new_state = tuple(sorted(new_state))
                outcomes.append((new_state, total_prob, achieved_goals))
    
    return outcomes

def getAllTransitionProbabilities(state, goalEdges, pSwap, pGen, maxAge):
    """
    Returns:
        List of (new_state, probability, achieved_goals) tuples
    """
    all_transitions = []
    
    # Get all possible swap action outcomes
    swap_outcomes = generateAllSwappingOutcomes(state, goalEdges, pSwap)
    
    # For each swap outcome, generate all possible generation outcomes
    for swap_state, swap_prob, achieved_goals in swap_outcomes:
        # Skip if swap probability is 0
        if swap_prob == 0:
            continue
            
        # First age the state
        aged_state = getAgedStates(swap_state, maxAge)
        
        # Then get all possible generation outcomes
        gen_outcomes = generateAllOutcomes(aged_state, pGen)
        
        # Combine the probabilities and add to transitions
        for final_state, gen_prob in gen_outcomes:
            # Skip if either probability is 0
            if gen_prob == 0:
                continue
                
            all_transitions.append((
                final_state,
                swap_prob * gen_prob,
                achieved_goals
            ))
    
    return all_transitions

def generateAllStateTransitions(edges, goalEdges, pSwap, pGen, maxAge):
    """
    Returns:
        Dictionary mapping state -> list of (next_state, prob, achieved_goals)
    """
    # Get all possible states
    all_states = getPossibleStates(edges, maxAge)
    transitions = {}
    
    # For each state, calculate all possible transitions
    for state in all_states:
        transitions[state] = getAllTransitionProbabilities(state, goalEdges, pSwap, pGen, maxAge)
    
    return transitions


In [ ]:
def getImmediateReward(achieved_goals, goal_success_counts, total_timesteps):
    if achieved_goals is None:
        return 0

    total_reward = 0
    for goal_edge, success in achieved_goals:
        if success:
            start, end = goal_edge
            num_edges = abs(end - start)
            instant_rate = pSwap ** (num_edges - 1)

            edr = max(0.0001, goal_success_counts[goal_edge] / max(1, total_timesteps))

            if instant_rate > 0 and edr > 0:
                total_reward += instant_rate / edr
    return total_reward

def calculate_fairness_metrics(edrs):
    """Calculate various fairness metrics for the given EDRs"""
    if not edrs:
        return 0, 0, 0
    
    values = list(edrs.values())
    
    # Jain's Fairness Index
    numerator = sum(values) ** 2
    denominator = len(values) * sum(x * x for x in values)
    jains_index = numerator / denominator if denominator != 0 else 0
    
    # Min-Max Ratio
    min_val = min(values)
    max_val = max(values)
    min_max_ratio = min_val / max_val if max_val != 0 else 0
    
    # Coefficient of Variation (CV)
    mean = np.mean(values)
    std = np.std(values)
    cv = std / mean if mean != 0 else 0
    
    return jains_index, min_max_ratio, cv

class QuantumNetworkQLearning:
    def __init__(self, edges, goalEdges, all_transitions, pSwap, pGen, maxAge):
        self.edges = edges
        self.goalEdges = goalEdges
        self.all_transitions = all_transitions
        self.pSwap = pSwap
        self.pGen = pGen
        self.maxAge = maxAge
        
        # Q-learning parameters
        self.alpha = 0.1
        self.gamma = 0.98
        self.epsilon = 0.3
        
        # Initialize Q-table using your transition structure
        self.Q = defaultdict(lambda: defaultdict(float))
        
        # Tracking variables
        self.goal_success_counts = {goal: 0 for goal in goalEdges}
        self.edr_history = {goal: [] for goal in goalEdges}
        self.reward_history = []
        
        # New convergence tracking variables
        self.episode_avg_q_values = []
        self.episode_avg_rewards = []
        self.episode_edrs = {goal: [] for goal in goalEdges}
        
        # Fairness tracking
        self.fairness_history = {
            'jains_index': [],
            'min_max_ratio': [],
            'cv': []
        }
        self.episode_fairness = {
            'jains_index': [],
            'min_max_ratio': [],
            'cv': []
        }
        
    def group_transitions_by_action(self, state_transitions):
        """Group transitions by the attempted goals, ignoring success/failure"""
        action_groups = defaultdict(list)
        
        for next_state, prob, achieved_goals in state_transitions:
            if achieved_goals is None:
                action_key = None
            else:
                action_key = tuple(sorted(goal for goal, _ in achieved_goals))
            action_groups[action_key].append((next_state, prob, achieved_goals))
        
        return action_groups
    
    def choose_action(self, state, training=True):
        """Choose action using epsilon-greedy"""
        action_groups = self.group_transitions_by_action(self.all_transitions[state])
        
        if training and random.random() < self.epsilon:
            # Explore: random action group
            action_keys = list(action_groups.keys())
            chosen_key = random.choice(action_keys)
            transitions = action_groups[chosen_key]
        else:
            # Exploit: best action group based on Q-values
            best_value = float('-inf')
            best_transitions = None
            
            for action_key, transitions in action_groups.items():
                # Get average Q-value for this action group
                q_value = np.mean([self.Q[state][next_state] for next_state, _, _ in transitions])
                if q_value > best_value:
                    best_value = q_value
                    best_transitions = transitions
            
            transitions = best_transitions
        
        # Choose specific transition based on probabilities
        probs = [t[1] for t in transitions]
        return random.choices(transitions, weights=probs)[0]
    
    def train(self, num_episodes=1000, max_steps=100):
        window_size = 100  # For moving averages
        
        for episode in range(num_episodes):
            state = tuple((edge, -1) for edge in self.edges)
            episode_reward = 0
            total_timesteps = 1
            episode_q_values = []
            self.goal_success_counts = {goal: 0 for goal in self.goalEdges}
            
            # Reset episode fairness tracking
            episode_fairness_values = {
                'jains_index': [],
                'min_max_ratio': [],
                'cv': []
            }
            
            for step in range(max_steps):
                # Choose and execute action
                next_state, prob, achieved_goals = self.choose_action(state, training=True)
                
                # Update goal successes
                if achieved_goals is not None:
                    for goal_edge, success in achieved_goals:
                        if success:
                            self.goal_success_counts[goal_edge] += 1
                
                # Get reward
                reward = getImmediateReward(achieved_goals, self.goal_success_counts, total_timesteps)
                episode_reward += reward
                
                # Q-learning update
                old_q = self.Q[state][next_state]
                next_q = max([self.Q[next_state][ns] for ns, _, _ in self.all_transitions[next_state]], default=0)
                self.Q[state][next_state] = old_q + self.alpha * (reward + self.gamma * next_q - old_q)
                
                # Track Q-values for convergence
                episode_q_values.append(self.Q[state][next_state])
                
                # Calculate current EDRs and fairness metrics
                current_edrs = {goal: self.goal_success_counts[goal] / total_timesteps for goal in self.goalEdges}
                jains, minmax, cv = calculate_fairness_metrics(current_edrs)
                
                # Store fairness metrics for this step
                episode_fairness_values['jains_index'].append(jains)
                episode_fairness_values['min_max_ratio'].append(minmax)
                episode_fairness_values['cv'].append(cv)
                
                # Update EDR tracking
                for goal in self.goalEdges:
                    current_edr = self.goal_success_counts[goal] / total_timesteps
                    if episode == num_episodes - 1:  # Only track last episode
                        self.edr_history[goal].append(current_edr)
                
                # Transition
                state = next_state
                total_timesteps += 1
            
            # Store episode metrics
            self.reward_history.append(episode_reward)
            self.episode_avg_q_values.append(np.mean(episode_q_values))
            
            # Store EDRs for each goal at the end of episode
            for goal in self.goalEdges:
                self.episode_edrs[goal].append(self.goal_success_counts[goal] / total_timesteps)
            
            # Store average fairness metrics for this episode
            self.fairness_history['jains_index'].append(np.mean(episode_fairness_values['jains_index']))
            self.fairness_history['min_max_ratio'].append(np.mean(episode_fairness_values['min_max_ratio']))
            self.fairness_history['cv'].append(np.mean(episode_fairness_values['cv']))
            
            # Calculate moving averages for reward
            if len(self.reward_history) >= window_size:
                avg_reward = np.mean(self.reward_history[-window_size:])
                self.episode_avg_rewards.append(avg_reward)
            
            if (episode + 1) % 100 == 0:
                print('episode', episode + 1)
    
    def plot_results(self):
        fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(15, 18))
        
        # Plot 1: EDR evolution (last episode)
        for goal, edr_values in self.edr_history.items():
            ax1.plot(range(len(edr_values)), edr_values, label=f'Goal {goal}')
        ax1.set_xlabel('Step')
        ax1.set_ylabel('EDR')
        ax1.set_title('EDR Evolution (Last Episode)')
        ax1.grid(True)
        ax1.legend()
        ax1.set_ylim(0, 1)
        
        # Plot 2: Reward history
        ax2.plot(self.reward_history, alpha=0.3, label='Raw')
        if len(self.episode_avg_rewards) > 0:
            ax2.plot(range(99, len(self.episode_avg_rewards) + 99), 
                    self.episode_avg_rewards, 
                    label='100-episode moving avg')
        ax2.set_xlabel('Episode')
        ax2.set_ylabel('Total Reward')
        ax2.set_title('Training Progress (Reward)')
        ax2.grid(True)
        ax2.legend()
        
        # Plot 3: Average Q-value evolution
        ax3.plot(self.episode_avg_q_values)
        ax3.set_xlabel('Episode')
        ax3.set_ylabel('Average Q-value')
        ax3.set_title('Q-value Convergence')
        ax3.grid(True)
        
        # Plot 4: EDR evolution across episodes
        for goal, edr_values in self.episode_edrs.items():
            ax4.plot(range(len(edr_values)), edr_values, label=f'Goal {goal}')
        ax4.set_xlabel('Episode')
        ax4.set_ylabel('EDR')
        ax4.set_title('EDR Evolution Across Episodes')
        ax4.grid(True)
        ax4.legend()
        ax4.set_ylim(0, 1)
        
        # Plot 5: Jain's Fairness Index evolution
        ax5.plot(self.fairness_history['jains_index'], label="Jain's Index")
        ax5.set_xlabel('Episode')
        ax5.set_ylabel('Fairness Index')
        ax5.set_title("Jain's Fairness Index Evolution")
        ax5.grid(True)
        ax5.set_ylim(0, 1)
        
        # Plot 6: Other fairness metrics
        ax6.plot(self.fairness_history['min_max_ratio'], label='Min-Max Ratio')
        ax6.plot(self.fairness_history['cv'], label='CV')
        ax6.set_xlabel('Episode')
        ax6.set_ylabel('Metric Value')
        ax6.set_title('Fairness Metrics Evolution')
        ax6.grid(True)
        ax6.legend()
        
        plt.tight_layout()
        plt.show()
        
        # Print convergence statistics
        print("\n=== Convergence Statistics ===")
        window_sizes = [100, 500, 1000]
        for window in window_sizes:
            if len(self.reward_history) >= window:
                print(f"\nLast {window} episodes:")
                print(f"Average reward: {np.mean(self.reward_history[-window:]):.2f}")
                print(f"Reward std dev: {np.std(self.reward_history[-window:]):.2f}")
                print(f"Average Q-value: {np.mean(self.episode_avg_q_values[-window:]):.2f}")
                print("\nFinal EDRs:")
                for goal in self.goalEdges:
                    print(f"Goal {goal}: {np.mean(self.episode_edrs[goal][-window:]):.4f}")
                print("\nFinal Fairness Metrics:")
                print(f"Jain's Index: {np.mean(self.fairness_history['jains_index'][-window:]):.4f}")
                print(f"Min-Max Ratio: {np.mean(self.fairness_history['min_max_ratio'][-window:]):.4f}")
                print(f"CV: {np.mean(self.fairness_history['cv'][-window:]):.4f}")

# Example usage
edges = [(0, 1), (1, 2), (2, 3), (3,4)]
goalEdges = [(0, 2), (1, 4)]
pSwap = 0.8
pGen = 0.8
maxAge = 2

# First generate all transitions using your existing code
all_transitions = generateAllStateTransitions(edges, goalEdges, pSwap, pGen, maxAge)

# Create and train agent
agent = QuantumNetworkQLearning(
    edges=edges,
    goalEdges=goalEdges,
    all_transitions=all_transitions,
    pSwap=pSwap,
    pGen=pGen,
    maxAge=maxAge
)
agent.train(num_episodes=100000, max_steps=3000)
agent.plot_results()

# Print final statistics
print("\n=== Final Statistics ===")
print(f"Parameters: pSwap={pSwap}, pGen={pGen}, maxAge={maxAge}")
print(f"Network: edges={edges}, goals={goalEdges}")

print("\nFinal EDRs (Last Episode):")
total_timesteps = len(next(iter(agent.edr_history.values())))
for goal in agent.goalEdges:
    final_edr = agent.goal_success_counts[goal] / total_timesteps
    print(f"Goal {goal}: EDR = {final_edr:.6f}")
    print(f"  - Successes: {agent.goal_success_counts[goal]}")
    print(f"  - Total timesteps: {total_timesteps}")

print(f"\nFinal Average Reward (last 100 episodes): {np.mean(agent.reward_history[-100:]):.4f}")

# Print final fairness metrics
final_edrs = {goal: agent.goal_success_counts[goal] / total_timesteps for goal in agent.goalEdges}
jains, minmax, cv = calculate_fairness_metrics(final_edrs)
print("\nFinal Fairness Metrics (Last Episode):")
print(f"Jain's Fairness Index: {jains:.4f}")
print(f"Min-Max Ratio: {minmax:.4f}")
print(f"Coefficient of Variation: {cv:.4f}")

episode 100
episode 200
episode 300
episode 400
episode 500
episode 600
episode 700
episode 800
episode 900
episode 1000
episode 1100
episode 1200
episode 1300
episode 1400
episode 1500
episode 1600
episode 1700
episode 1800
episode 1900
episode 2000
episode 2100
episode 2200
episode 2300
episode 2400
episode 2500
episode 2600
episode 2700
episode 2800
episode 2900
episode 3000
episode 3100
episode 3200
episode 3300
episode 3400
episode 3500
episode 3600
episode 3700
episode 3800
episode 3900
episode 4000
episode 4100
episode 4200
episode 4300
episode 4400
episode 4500
episode 4600
episode 4700
episode 4800
episode 4900
episode 5000
episode 5100
episode 5200
episode 5300
episode 5400
episode 5500
episode 5600
episode 5700
episode 5800
episode 5900
episode 6000
episode 6100
episode 6200
episode 6300
episode 6400
episode 6500
episode 6600
episode 6700
episode 6800
episode 6900
episode 7000
episode 7100
episode 7200
episode 7300
episode 7400
episode 7500
episode 7600
episode 7700
episode 